<img src="http://193.175.187.164/static/OEP_logo_2_no_text.svg" alt="OpenEnergy Platform" height="100" width="100"  align="left"/>

# OpenEnergyPlatform
<br>
## Usage of OpenEnergyPlatform API-Dialect (oedialect)
Repository: https://github.com/openego/oedialect

In [ ]:
__copyright__ = "Reiner Lemoine Institut"
__license__   = "GNU Affero General Public License Version 3 (AGPL-3.0)"
__url__       = "https://github.com/openego/data_processing/blob/master/LICENSE"
__author__    = "henhuy, Ludee, nesnoj"

In [ ]:
import pandas as pd
import getpass
import sqlalchemy as sa
from sqlalchemy.orm import sessionmaker
import oedialect

In [ ]:
# Whitespaces are not a problem for setting up the url!
user = input('Enter OEP-username:')
token = getpass.getpass('Token:')

In [1]:
# Create Engine:
OEP_URL = 'oep.iks.cs.ovgu.de'
OED_STRING = 'postgresql+oedialect://{user}:{token}@{OEP_URL}'.format(user=user,
                                                                      token=token,
                                                                      OEP_URL=OEP_URL)

engine = sa.create_engine(OED_STRING)
metadata = sa.MetaData(bind=engine)

## Setup a Table

In [ ]:
table_name = 'example_dialect_table'
schema_name = 'sandbox'

ExampleTable = sa.Table(
    table_name,
    metadata,
    sa.Column('name', sa.VARCHAR(50)),
    sa.Column('age', sa.INTEGER),
    sa.Column('stadtname', sa.VARCHAR(50)),
    schema=schema_name
)

## Create the new Table

In [ ]:
conn = engine.connect()
print('Connection established')
if not engine.dialect.has_table(conn, table_name, schema_name):
    ExampleTable.create()
    print('Created table')
else:
    print('Table already exists')

## Insert data into Table

In [ ]:
Session = sessionmaker(bind=engine)
session = Session()
try:
    insert_statement = ExampleTable.insert().values(
        [
            dict(name='Peter', age=25),
            dict(name='Inge', age=42),
            dict(name='Horst', age=69)
        ]
    )
    session.execute(insert_statement)
    session.commit()
    print('Insert successful!')
except Exception as e:
    session.rollback()
    raise
    print('Insert incomplete!')
finally:
    session.close()

## Select from Table

In [ ]:
Session = sessionmaker(bind=engine)
session = Session()
print(session.query(ExampleTable).all())
session.close()

In [ ]:
Session = sessionmaker(bind=engine)
session = Session()
df = pd.DataFrame(session.query(ExampleTable).all())
session.close()
df

## Update data in Table

In [ ]:
# ????